CAMS

- mean, median, std, min, max for PM2.5 values by year and year-month

- maps for both results


In [4]:
import xarray as xr
import pandas as pd
import numpy as np
import os

In [6]:
# Define the folder containing your NC files
folder_path = r"E:\IPMA\CAMS\chem_singlvl\5conversion_2003_2024"

# Initialize a list to store results
results = []

# Loop through all the NC files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".nc"):
        # Load the NC file using xarray
        file_path = os.path.join(folder_path, file_name)
        ds = xr.open_dataset(file_path)
        
        # Extract the time variable, assuming it's already in '2003-01-01' format
        time = pd.to_datetime(ds['time'].values)  # Directly convert to pandas datetime
        
        # Extract the year directly from the datetime object
        years = time.year  # Extract the year directly from the datetime object
        
        # Extract the 'pm10' variable (assuming it's in the dataset)
        pm10 = ds['pm10'].values
        
        # Create a DataFrame for the current file
        df = pd.DataFrame({
            'latitude': ds['latitude'].values.flatten(),
            'longitude': ds['longitude'].values.flatten(),
            'time': time,
            'pm10': pm10.flatten()
        })
        
        # Add the 'year' column to the DataFrame
        df['year'] = years
        
        # Group the data by the year extracted from the time variable
        df_year = df.groupby('year')
        
        # Calculate statistics for 'pm10' for each year
        for year, group in df_year:
            stats = {
                'year': year,
                'mean_pm10': group['pm10'].mean(),
                'median_pm10': group['pm10'].median(),
                'std_pm10': group['pm10'].std(),
                'max_pm10': group['pm10'].max(),
                'min_pm10': group['pm10'].min(),
            }
            results.append(stats)

# Create a DataFrame with the results
stats_df = pd.DataFrame(results)

# Display the results
print(stats_df)

ValueError: All arrays must be of the same length